In [ ]:
!pip install scipy

In [78]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
from scipy import stats
import plotly.express as px

In [73]:
np.random.seed(234)
seeds = np.random.randint(1, 999, 20)

In [97]:
data=None
for seed in seeds:
    with open(f"results/Thompson_{seed}.npy", 'rb') as f:
        arr = np.load(f).reshape(1, -1)
        data = np.append(data, arr, axis=0) if data is not None else arr
data = pd.DataFrame(data).T
regret = data.expanding().mean()
regret.index = regret.index * 3

N = regret.shape[1]
sqrtN = np.sqrt(N)
D = stats.t.ppf(.975, df=N-1)
regret_stats = regret.agg(['mean', 'std'], axis=1)
regret_stats['UB'] = regret_stats.eval('mean + @D * std / @sqrtN')
regret_stats['LB'] = regret_stats.eval('mean - @D * std / @sqrtN')

fig = go.Figure([
    go.Scatter(
        name='Thompson Regret',
        x=regret_stats.index,
        y=regret_stats['mean'],
        mode='lines',
        line=dict(color=px.colors.DEFAULT_PLOTLY_COLORS[2]),
    ),
    go.Scatter(
        name='Upper Bound',
        x=regret_stats.index,
        y=regret_stats.UB,
        mode='lines',
        #marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=regret_stats.index,
        y=regret_stats.LB,
        #marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor=px.colors.DEFAULT_PLOTLY_COLORS[2].replace('b', 'ba').replace(')', ', .3)'),
        fill='tonexty',
        showlegend=False
    )
])
fig = fig.update_layout(width=1000, height=700, title='Expected Cumulative Regret', 
                       yaxis={'title': 'Expected Regret', 'range': (0, 0.6)}, xaxis={'title': 'Sample'})
plotly.offline.plot(fig, filename = 'plot.html', auto_open=False)

'plot.html'

In [99]:
fig = go.Figure()

for i, (k, kk) in enumerate(list(zip(['UCB', 'Neural', 'Thompson'], ['LinUCB', 'NeuralLinUCB', 'Thompson']))):
    data=None
    for seed in seeds:
        with open(f"results/{k}_{seed}.npy", 'rb') as f:
            arr = np.load(f).reshape(1, -1)
            data = np.append(data, arr, axis=0) if data is not None else arr
    data = pd.DataFrame(data).T
    regret = data.expanding().mean()
    regret.index = regret.index * 3

    N = regret.shape[1]
    sqrtN = np.sqrt(N)
    D = stats.t.ppf(.975, df=N-1)
    regret_stats = regret.agg(['mean', 'std'], axis=1)
    regret_stats['UB'] = regret_stats.eval('mean + @D * std / @sqrtN')
    regret_stats['LB'] = regret_stats.eval('mean - @D * std / @sqrtN')



    fig = fig.add_trace(go.Scatter(
            name=f'{kk} Regret',
            x=regret_stats.index,
            y=regret_stats['mean'],
            mode='lines',
            line=dict(color=px.colors.DEFAULT_PLOTLY_COLORS[i]),
        ))
    fig = fig.add_trace(go.Scatter(
            name='Upper Bound',
            x=regret_stats.index,
            y=regret_stats.UB,
            mode='lines',
            #marker=dict(color="#444"),
            line=dict(width=0),
            showlegend=False
        ))
    fig = fig.add_trace(go.Scatter(
            name='Lower Bound',
            x=regret_stats.index,
            y=regret_stats.LB,
            #marker=dict(color="#444"),
            line=dict(width=0),
            mode='lines',
            fillcolor=px.colors.DEFAULT_PLOTLY_COLORS[i].replace('b', 'ba').replace(')', ', .3)'),
            fill='tonexty',
            showlegend=False
        ))

fig = fig.update_layout(width=1000, height=700, title='Expected Cumulative Regret', 
                           yaxis={'title': 'Expected Regret', 'range': (0, 0.6)}, xaxis={'title': 'Sample'})

plotly.offline.plot(fig, filename = 'plot.html', auto_open=False)

'plot.html'